# Corporación Favorita Grocery Sales Forecasting

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [ ]:
from fastai import *
from fastai.tabular import *

from IPython.display import display
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

from pandas.api.types import is_string_dtype, is_numeric_dtype

In [3]:
import os
os.getcwd()

'/mnt/e/ubuntu/Projects/Forecasting'

In [4]:
os.listdir()

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'data',
 'README.md',
 'Sales_Forecasting.ipynb']

In [5]:
PATH="data/grocery-sales/"
!ls {PATH}

holidays_events.csv  oil.csv		    stores.csv	train.csv
items.csv	     sample_submission.csv  test.csv	transactions.csv


## Read Data

In [6]:
!head data/grocery-sales/train.csv

id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2013-01-01,25,103665,7.0,
1,2013-01-01,25,105574,1.0,
2,2013-01-01,25,105575,2.0,
3,2013-01-01,25,108079,1.0,
4,2013-01-01,25,108701,1.0,
5,2013-01-01,25,108786,3.0,
6,2013-01-01,25,108797,1.0,
7,2013-01-01,25,108952,1.0,
8,2013-01-01,25,111397,13.0,


In [7]:
types = {"id": "int64",
         "item_nbr":"int32",
         "store_nbr":"int8",
         "unit_sales":"float32",
         "onpromotion":"object"}

In [10]:
%%time
df_all=pd.read_csv(f"{PATH}train.csv",parse_dates=["date"],dtype=types,
                  infer_datetime_format=True)

CPU times: user 1min 47s, sys: 36.8 s, total: 2min 24s
Wall time: 2min 27s


In [11]:
df_all.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN


In [13]:
df_all["onpromotion"].fillna(False,inplace=True)
df_all["onpromotion"]=df_all["onpromotion"].map({"False":False,"True":True})
df_all["onpromotion"]=df_all["onpromotion"].astype(bool)

%time df_all.to_feather("tmp/raw_groceries")

CPU times: user 2.38 s, sys: 53.5 s, total: 55.9 s
Wall time: 1min 13s


In [14]:
%time df_all.describe(include="all")

CPU times: user 22.9 s, sys: 22.6 s, total: 45.5 s
Wall time: 47.1 s


,id,date,store_nbr,item_nbr,unit_sales,onpromotion
count,1.254970e+08,125497040,1.254970e+08,1.254970e+08,1.254970e+08,125497040
unique,NaN,1684,NaN,NaN,NaN,1
top,NaN,2017-07-01 00:00:00,NaN,NaN,NaN,True
freq,NaN,118194,NaN,NaN,NaN,125497040
first,NaN,2013-01-01 00:00:00,NaN,NaN,NaN,NaN
last,NaN,2017-08-15 00:00:00,NaN,NaN,NaN,NaN
mean,6.274852e+07,NaN,2.746458e+01,9.727692e+05,5.319669e+00,NaN
std,3.622788e+07,NaN,1.633051e+01,5.205336e+05,2.306714e+01,NaN
min,0.000000e+00,NaN,1.000000e+00,9.699500e+04,-1.537200e+04,NaN
25%,3.137426e+07,NaN,1.200000e+01,5.223830e+05,2.000000e+00,NaN


In [16]:
df_test=pd.read_csv(f"{PATH}test.csv",parse_dates=["date"],dtype=types,
                    infer_datetime_format=True)
df_test["onpromotion"].fillna(False,inplace=True)                    
df_test["onpromotion"]=df_test["onpromotion"].map({"False":False,"True":True})
df_test["onpromotion"]=df_test["onpromotion"].astype(bool)
df_test.describe(include="all")
                    

,id,date,store_nbr,item_nbr,onpromotion
count,3.370464e+06,3370464,3.370464e+06,3.370464e+06,3370464
unique,NaN,16,NaN,NaN,2
top,NaN,2017-08-27 00:00:00,NaN,NaN,False
freq,NaN,210654,NaN,NaN,3171867
first,NaN,2017-08-16 00:00:00,NaN,NaN,NaN
last,NaN,2017-08-31 00:00:00,NaN,NaN,NaN
mean,1.271823e+08,NaN,2.750000e+01,1.244798e+06,NaN
std,9.729693e+05,NaN,1.558579e+01,5.898362e+05,NaN
min,1.254970e+08,NaN,1.000000e+00,9.699500e+04,NaN
25%,1.263397e+08,NaN,1.400000e+01,8.053210e+05,NaN


In [17]:
df_all.tail()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
125497035,125497035,2017-08-15,54,2089339,4.0,True
125497036,125497036,2017-08-15,54,2106464,1.0,True
125497037,125497037,2017-08-15,54,2110456,192.0,True
125497038,125497038,2017-08-15,54,2113914,198.0,True
125497039,125497039,2017-08-15,54,2116416,2.0,True


In [20]:
df_all=pd.read_feather("tmp/raw_groceries")

In [21]:
df_all["unit_sales"]=np.log1p(np.clip(df_all["unit_sales"],0,None))

In [22]:
%time add_datepart(df_all,"date")

CPU times: user 4min 30s, sys: 1min 52s, total: 6min 23s
Wall time: 8min 54s


,id,store_nbr,item_nbr,unit_sales,onpromotion,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,0,25,103665,2.079442,True,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
1,1,25,105574,0.693147,True,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
2,2,25,105575,1.098612,True,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
3,3,25,108079,0.693147,True,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
4,4,25,108701,0.693147,True,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125497035,125497035,54,2089339,1.609438,True,2017,8,33,15,1,227,False,False,False,False,False,False,1502755200
125497036,125497036,54,2106464,0.693147,True,2017,8,33,15,1,227,False,False,False,False,False,False,1502755200
125497037,125497037,54,2110456,5.262690,True,2017,8,33,15,1,227,False,False,False,False,False,False,1502755200
125497038,125497038,54,2113914,5.293305,True,2017,8,33,15,1,227,False,False,False,False,False,False,1502755200


In [23]:
def split_vals(a,n):return a[:n].copy(),a[n:].copy()

In [25]:
n_valid=len(df_test)
n_trn=len(df_all)-n_valid
train,valid=split_vals(df_all,n_trn)
train.shape,valid.shape

((122126576, 18), (3370464, 18))

In [26]:
# train_cats(raw_train)
# apply_cats(raw_valid,raw_train)

In [ ]:
%%time
trn,y,nas=proc_df(train,"unit_sales")
val,y_val=proc_df(valid,"unit_sales")

### Models

In [27]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res=[rmse(m.predict(x),y),rmse(m.predict(val),y_val),
        m.score(x,y),m.score(val,y_val)]
    if hasattr (m,"oob_score"):res.append(m.oob_score_)
    print(res)